<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/deagwon/Dacon_submittion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import gc
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import stats
warnings.filterwarnings("ignore")
# style 설정
plt.style.use('dark_background')
plt.style.use('seaborn-colorblind')

# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 모델만들기

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

## 데이터 제네레이터 만들기

In [0]:
# GMI좌표로 보정된 데이터
gmi_pre = np.load('/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci_32bits.npy')

In [0]:
model.save('model.h5')

## submission 만들기

In [0]:
#test data도 시간 데이터 추가하기
test = np.load('/content/drive/My Drive/2020 Kaggle Study/data/test.npy')
test_orbit = np.load('/content/drive/My Drive/2020 Kaggle Study/data/orbit_no_shuffle/test_orbit.npy')
test_orbit = pd.DataFrame(test_orbit)
test_orbit.columns = ['orbit_number', 'order']
test_orbit['year'] = 2019
test_orbit['month'] = (test_orbit['orbit_number'] - 27510) / 5682 * 12
test_orbit['day'] = (test_orbit['orbit_number'] - 27510)/5682 * 365
test_orbit['time'] = (test_orbit['orbit_number'] - 27510)/(5682)*(365 * 24) % 24


test_orbit.month = test_orbit.month.apply(np.ceil)# 올림
test_orbit.month[test_orbit.month == 0] = 1 # 가장 처음 나오는 orbit은 1월로
test_orbit.day = test_orbit.day.apply(np.ceil)# 올림
test_orbit.day[test_orbit.day == 0] = 1 # 가장 처음 나오는 orbit은 1일로
test_orbit.time = test_orbit.time.apply(np.floor)# 내림

#1. month_sin, month_cos만들기
test_orbit['month_sin'] = np.sin(2 * np.pi * (test_orbit['month']-1)/11)
test_orbit['month_cos'] = np.cos(2 * np.pi * (test_orbit['month']-1)/11)

# 2. day
test_orbit['day_sin'] = np.sin(2 * np.pi * (test_orbit['day']-1)/365)
test_orbit['day_cos'] = np.cos(2 * np.pi * (test_orbit['day']-1)/365)

# 3. time
test_orbit['time_sin'] = np.sin(2 * np.pi * (test_orbit['time'])/23)
test_orbit['time_cos'] = np.cos(2 * np.pi * (test_orbit['time'])/23)

In [5]:
#스케일러 학습
from sklearn.preprocessing import RobustScaler
path = '/content/drive/My Drive/2020 Kaggle Study/deagwon/ch_part_eda/'
robust_sc = {}
for ch in tqdm(range(9)):
    for sw in range(2):
        for dn in range(2):
            for ls in range(2):
                for la in range(4): 
                    data = np.load(f'{path}ch{ch}_{sw}{dn}{ls}{la}.npy')
                    robust_scaler = RobustScaler()
                    robust_scaler.fit(data.reshape(-1,1))
                    robust_sc[ch, sw, dn, ls, la] = robust_scaler
                    #pickle.dump(ch, model, open(f'{path}scaler_{ch}{sw}{dn}{ls}{la}.sav', 'wb'))

100%|██████████| 9/9 [04:43<00:00, 31.51s/it]


In [6]:
# land type 보정
def re_landtype(data):
    rain = data[:,:,:,9].reshape(-1)
    rain = rain//100
    #data[data[:,:,:,9] == 0] = 0 #Ocean
    rain[rain == 1] = 1 #land
    rain[rain == 2] = 0.5 #costal
    rain[rain == 3] = 0.2 #in land water
    data[:,:,:,9] = rain.reshape(-1,40,40)
    return data
test = re_landtype(test)

test_day_time = test_orbit[['day', 'time']].values
test_normalization = np.zeros_like(test[:,:,:,:10], np.float32)# target 제거 총 10개의 feature

#위도의 최대 최소
lat_max = 55.846363#np.max(test[:,:,:,11].reshape(-1))
lat_min = 4.0785685#np.min(test[:,:,:,11].reshape(-1))
day_start = int(7/2)
day_end = int((23+7)/2)
summer_start = int((201 + 50) / 2)
summer_end = int((50 + (365 - 201))/2 + 201)


land_type = test[:,:,:,9].reshape(-1)
land_type = land_type>0.1

length = len(test[:,:,:,0].reshape(-1))
for ch in range(9):
    for sample_index in tqdm(range(len(test))):
        normalization_sample = np.zeros((1600,1))
        summer_winter = ((test_day_time[sample_index,0] < summer_end) & (test_day_time[sample_index,0] > summer_start)) * 1
        day_night = ((test_day_time[sample_index,1] < day_end) & (test_day_time[sample_index,1] > day_start)) * 1
        lat = int((test[sample_index,20,20,11]-lat_min)*4/(lat_max-lat_min))
        land_type_index = land_type[sample_index * 1600 : (sample_index+1) * 1600]

        if land_type_index.sum() != 0:
            normalization_sample[land_type_index] = robust_sc[ch,summer_winter,
                                                              day_night,1,lat].transform(test[sample_index,:,:,ch].reshape(-1,1)[land_type_index])
        if sum(land_type_index != True) != 0:
            normalization_sample[land_type_index != True] = robust_sc[ch,summer_winter,
                                                                      day_night,0,lat].transform(test[sample_index,:,:,ch].reshape(-1,1)[land_type_index != True])
        
        test_normalization[sample_index,:,:,ch] = normalization_sample.reshape(40,40)
        
test_normalization[:,:,:,9] = test[:,:,:,9]
#test_normalization[:,:,:,10] = test[:,:,:,-1]

100%|██████████| 2416/2416 [00:07<00:00, 320.23it/s]


In [7]:
test_normalization.shape

(2416, 40, 40, 10)

In [0]:
model = tf.keras.models.load_model('model_4.h5', compile = False)
model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
pred = model.predict(test_normalization)
submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')

In [0]:
submission.iloc[:,1:] = pred.reshape(-1, 1600)

In [0]:
submission.to_csv('Dacon_normalization.csv', index = False)

In [0]:
# 제출파일 구글 드라이브에 올리기
!cp Dacon_landtype.csv '/content/drive/My Drive/2020 Kaggle Study/submission'